# M4 Forecasting Competition with `sktime` Catalogues

This notebook demonstrates how to reproduce the classical benchmarks from the M4 forecasting competition using sktime’s catalogue and benchmarking framework.

Instead of manually wiring datasets, forecasters, and metrics, sktime provides catalogues: reusable, declarative collections of benchmarking components.

For the M4 competition, each temporal granularity (hourly, daily, weekly, monthly, quarterly, yearly) is represented by its own catalogue.

In this notebook, we gather all M4 catalogues, add them to benchmarking objects, and run the benchmarks end-to-end.

## Imports

In [ ]:
from sktime.benchmarking.forecasting import ForecastingBenchmark
from sktime.catalogues import (
    M4CompetitionCatalogueDaily,
    M4CompetitionCatalogueHourly,
    M4CompetitionCatalogueMonthly,
    M4CompetitionCatalogueQuarterly,
    M4CompetitionCatalogueWeekly,
    M4CompetitionCatalogueYearly,
)

## What is an M4 catalogue?

Each M4 catalogue:

• Binds a single M4 dataset (one temporal granularity)

• Defines a shared set of classical forecasters

• Uses Overall Weighted Average (OWA) with the correct seasonal period (sp)

• Is fully compatible with sktime’s benchmarking framework

## Creating the benchmark

We first create `ForecastingBenchmark` objects for each temporal granularity. These objects will collect all benchmark components and coordinate execution.

In [ ]:
benchmarks = {
    "hourly": ForecastingBenchmark(),
    "daily": ForecastingBenchmark(),
    "weekly": ForecastingBenchmark(),
    "monthly": ForecastingBenchmark(),
    "quarterly": ForecastingBenchmark(),
    "yearly": ForecastingBenchmark(),
}

## Adding M4 catalogues

Each M4 catalogue is added to its own ForecastingBenchmark instance.

This is important because ForecastingBenchmark evaluates the Cartesian product of all added components.

Running one benchmark per catalogue ensures that datasets, forecasters, and metrics remain correctly aligned.

In [ ]:
benchmarks["hourly"].add(M4CompetitionCatalogueHourly())
benchmarks["daily"].add(M4CompetitionCatalogueDaily())
benchmarks["weekly"].add(M4CompetitionCatalogueWeekly())
benchmarks["monthly"].add(M4CompetitionCatalogueMonthly())
benchmarks["quarterly"].add(M4CompetitionCatalogueQuarterly())
benchmarks["yearly"].add(M4CompetitionCatalogueYearly())